# Query Analysis (QAN)

This notebook demonstrates how to analyze database query performance using data collected by Project Obsidian Core.

## Setup Connection to Druid

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sqlalchemy import create_engine
from datetime import datetime, timedelta

# Connect to Druid SQL
druid_url = "druid://druid-broker:8082/druid/v2/sql/"
engine = create_engine(druid_url)

# Test connection
try:
    with engine.connect() as conn:
        result = conn.execute("SELECT 1 AS test").fetchone()
    print(f"Connected to Druid successfully: {result}")
except Exception as e:
    print(f"Error connecting to Druid: {e}")

## Helper Functions for QAN Analysis

In [ ]:
def get_top_queries(db_system, metric_column, time_window_minutes=60, limit=10):
    """
    Get top queries based on a specific metric for a given database system and time window.
    
    Parameters:
    - db_system: 'mysql' or 'postgresql'
    - metric_column: Column to sort by (e.g., 'sum_total_exec_time', 'sum_rows_examined')
    - time_window_minutes: Time window in minutes (default: 60)
    - limit: Number of queries to return (default: 10)
    """
    end_time = datetime.now()
    start_time = end_time - timedelta(minutes=time_window_minutes)
    
    # Determine digest/id column based on db_system
    id_column = '"db.statement.digest"' if db_system == 'mysql' else '"db.query.id"'
    
    query = f"""
    SELECT 
      {id_column} AS query_id,
      "db.statement.sample" AS query_sample,
      "resource.instance.id" AS instance_id,
      SUM({metric_column}) AS total,
      SUM(sum_calls) AS calls,
      CASE WHEN SUM(sum_calls) > 0 THEN SUM({metric_column}) / SUM(sum_calls) ELSE 0 END AS avg_per_call
    FROM qan_db
    WHERE 
      "db.system" = '{db_system}' AND
      __time BETWEEN TIMESTAMP '{start_time.strftime('%Y-%m-%d %H:%M:%S')}' AND TIMESTAMP '{end_time.strftime('%Y-%m-%d %H:%M:%S')}'
    GROUP BY 1, 2, 3
    ORDER BY 4 DESC
    LIMIT {limit}
    """
    
    try:
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Error fetching top queries: {e}")
        return pd.DataFrame()

def format_sample(sample, max_length=80):
    """Format query sample for display"""
    if not sample or pd.isna(sample):
        return "[No sample available]"
        
    sample = str(sample).strip()
    if len(sample) > max_length:
        return sample[:max_length] + "..."
    return sample

def analyze_query_trend(db_system, query_id, metric_column, time_window_hours=24, interval_minutes=5):
    """
    Analyze trend of a specific query over time.
    
    Parameters:
    - db_system: 'mysql' or 'postgresql'
    - query_id: The digest or ID of the query
    - metric_column: Column to analyze (e.g., 'sum_total_exec_time', 'sum_rows_examined')
    - time_window_hours: Time window in hours (default: 24)
    - interval_minutes: Interval for time buckets in minutes (default: 5)
    """
    end_time = datetime.now()
    start_time = end_time - timedelta(hours=time_window_hours)
    
    # Determine id column based on db_system
    id_column = '"db.statement.digest"' if db_system == 'mysql' else '"db.query.id"'
    
    query = f"""
    SELECT 
      time_floor(__time, 'PT{interval_minutes}M') AS time_bucket,
      SUM({metric_column}) AS total,
      SUM(sum_calls) AS calls,
      CASE WHEN SUM(sum_calls) > 0 THEN SUM({metric_column}) / SUM(sum_calls) ELSE 0 END AS avg_per_call
    FROM qan_db
    WHERE 
      "db.system" = '{db_system}' AND
      {id_column} = '{query_id}' AND
      __time BETWEEN TIMESTAMP '{start_time.strftime('%Y-%m-%d %H:%M:%S')}' AND TIMESTAMP '{end_time.strftime('%Y-%m-%d %H:%M:%S')}'
    GROUP BY 1
    ORDER BY 1
    """
    
    try:
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Error fetching query trend: {e}")
        return pd.DataFrame()

## Top 10 MySQL Queries by Execution Time (Last Hour)

In [ ]:
# Get top 10 MySQL queries by execution time in the last hour
mysql_top_queries = get_top_queries('mysql', 'sum_total_exec_time', time_window_minutes=60, limit=10)

if not mysql_top_queries.empty:
    # Format for display
    display_df = mysql_top_queries.copy()
    display_df['query_sample'] = display_df['query_sample'].apply(format_sample)
    display_df['total_seconds'] = display_df['total'] / 1000000000  # Convert from nanoseconds to seconds
    display_df['avg_seconds'] = display_df['avg_per_call'] / 1000000000  # Convert from nanoseconds to seconds
    
    # Display table
    display(display_df[['query_sample', 'calls', 'total_seconds', 'avg_seconds', 'instance_id']].rename(
        columns={
            'query_sample': 'Query',
            'calls': 'Executions',
            'total_seconds': 'Total Time (s)',
            'avg_seconds': 'Avg Time (s)',
            'instance_id': 'Instance'
        }
    ))
    
    # Create bar chart
    fig = px.bar(
        display_df, 
        x='query_id', 
        y='total_seconds',
        hover_data=['query_sample', 'calls', 'avg_seconds'],
        labels={
            'query_id': 'Query ID',
            'total_seconds': 'Total Execution Time (s)',
            'query_sample': 'Query',
            'calls': 'Executions',
            'avg_seconds': 'Avg Time (s)'
        },
        title='Top 10 MySQL Queries by Execution Time (Last Hour)'
    )
    fig.update_layout(height=500)
    fig.show()
else:
    print("No MySQL query data found for the last hour.")

## Top 10 PostgreSQL Queries by Rows Examined (Last Hour)

In [ ]:
# Get top 10 PostgreSQL queries by rows examined in the last hour
postgres_top_queries = get_top_queries('postgresql', 'sum_rows_examined', time_window_minutes=60, limit=10)

if not postgres_top_queries.empty:
    # Format for display
    display_df = postgres_top_queries.copy()
    display_df['query_sample'] = display_df['query_sample'].apply(format_sample)
    
    # Display table
    display(display_df[['query_sample', 'calls', 'total', 'avg_per_call', 'instance_id']].rename(
        columns={
            'query_sample': 'Query',
            'calls': 'Executions',
            'total': 'Total Rows Examined',
            'avg_per_call': 'Avg Rows per Call',
            'instance_id': 'Instance'
        }
    ))
    
    # Create bar chart
    fig = px.bar(
        display_df, 
        x='query_id', 
        y='total',
        hover_data=['query_sample', 'calls', 'avg_per_call'],
        labels={
            'query_id': 'Query ID',
            'total': 'Total Rows Examined',
            'query_sample': 'Query',
            'calls': 'Executions',
            'avg_per_call': 'Avg Rows per Call'
        },
        title='Top 10 PostgreSQL Queries by Rows Examined (Last Hour)'
    )
    fig.update_layout(height=500)
    fig.show()
else:
    print("No PostgreSQL query data found for the last hour.")

## Query Trend Analysis for Specific Query

In [ ]:
# Select one of the top queries for trend analysis
if not mysql_top_queries.empty:
    # Get the first query ID from top queries
    selected_query_id = mysql_top_queries.iloc[0]['query_id']
    selected_query_sample = mysql_top_queries.iloc[0]['query_sample']
    
    print(f"Analyzing trend for query: {format_sample(selected_query_sample)}")
    
    # Get trend data for the last 24 hours with 5-minute intervals
    trend_df = analyze_query_trend('mysql', selected_query_id, 'sum_total_exec_time', time_window_hours=24, interval_minutes=5)
    
    if not trend_df.empty:
        # Convert time values to seconds
        trend_df['total_seconds'] = trend_df['total'] / 1000000000
        trend_df['avg_seconds'] = trend_df['avg_per_call'] / 1000000000
        
        # Create line chart for execution time trend
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=trend_df['time_bucket'],
            y=trend_df['total_seconds'],
            mode='lines',
            name='Total Time (s)'
        ))
        
        fig.add_trace(go.Scatter(
            x=trend_df['time_bucket'],
            y=trend_df['calls'],
            mode='lines',
            name='Executions',
            yaxis='y2'
        ))
        
        fig.update_layout(
            title=f"Query Execution Trend (Last 24 Hours)",
            xaxis_title="Time",
            yaxis_title="Total Time (s)",
            yaxis2=dict(
                title="Executions",
                overlaying="y",
                side="right"
            ),
            height=500,
            legend=dict(x=0.01, y=0.99, orientation='h')
        )
        fig.show()
        
        # Create line chart for average execution time trend
        fig2 = px.line(
            trend_df, 
            x="time_bucket", 
            y="avg_seconds",
            title="Average Execution Time per Call (Last 24 Hours)",
            labels={
                "time_bucket": "Time",
                "avg_seconds": "Avg Time per Call (s)"
            }
        )
        fig2.update_layout(height=400)
        fig2.show()
    else:
        print("No trend data found for the selected query.")
else:
    print("No queries available for trend analysis.")

## Compare Different Query Metrics

In [ ]:
# Function to get multiple top metrics
def get_top_by_multiple_metrics(db_system='mysql', time_window_minutes=60, limit=10):
    end_time = datetime.now()
    start_time = end_time - timedelta(minutes=time_window_minutes)
    
    # Determine id column based on db_system
    id_column = '"db.statement.digest"' if db_system == 'mysql' else '"db.query.id"'
    
    query = f"""
    WITH aggregated AS (
      SELECT 
        {id_column} AS query_id,
        "db.statement.sample" AS query_sample,
        SUM(sum_total_exec_time) AS exec_time,
        SUM(sum_rows_examined) AS rows_examined,
        SUM(sum_calls) AS calls,
        SUM(COALESCE(sum_temp_disk_tables, 0)) AS temp_disk_tables
      FROM qan_db
      WHERE 
        "db.system" = '{db_system}' AND
        __time BETWEEN TIMESTAMP '{start_time.strftime('%Y-%m-%d %H:%M:%S')}' AND TIMESTAMP '{end_time.strftime('%Y-%m-%d %H:%M:%S')}'
      GROUP BY 1, 2
    ),
    exec_time_rank AS (
      SELECT query_id, query_sample, exec_time, calls, rows_examined, temp_disk_tables,
             ROW_NUMBER() OVER (ORDER BY exec_time DESC) AS rank
      FROM aggregated
    ),
    rows_rank AS (
      SELECT query_id, query_sample, exec_time, calls, rows_examined, temp_disk_tables,
             ROW_NUMBER() OVER (ORDER BY rows_examined DESC) AS rank
      FROM aggregated
    ),
    temp_tables_rank AS (
      SELECT query_id, query_sample, exec_time, calls, rows_examined, temp_disk_tables,
             ROW_NUMBER() OVER (ORDER BY temp_disk_tables DESC) AS rank
      FROM aggregated
      WHERE temp_disk_tables > 0
    )
    SELECT 'Execution Time' AS metric, query_id, query_sample, exec_time AS value, calls, rows_examined, temp_disk_tables
    FROM exec_time_rank
    WHERE rank <= {limit}
    UNION ALL
    SELECT 'Rows Examined' AS metric, query_id, query_sample, rows_examined AS value, calls, exec_time, temp_disk_tables
    FROM rows_rank
    WHERE rank <= {limit}
    UNION ALL
    SELECT 'Temp Disk Tables' AS metric, query_id, query_sample, temp_disk_tables AS value, calls, exec_time, rows_examined
    FROM temp_tables_rank
    WHERE rank <= {limit}
    ORDER BY metric, value DESC
    """
    
    try:
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Error fetching top queries by multiple metrics: {e}")
        return pd.DataFrame()

# Get top MySQL queries by multiple metrics
multi_metric_df = get_top_by_multiple_metrics('mysql', time_window_minutes=60, limit=5)

if not multi_metric_df.empty:
    # Format for display
    multi_metric_df['query_sample'] = multi_metric_df['query_sample'].apply(format_sample)
    
    # Convert exec_time to seconds if needed
    exec_time_df = multi_metric_df[multi_metric_df['metric'] == 'Execution Time'].copy()
    if not exec_time_df.empty and exec_time_df['value'].max() > 1000000:
        # Values are likely in nanoseconds, convert to seconds
        exec_time_df['value'] = exec_time_df['value'] / 1000000000
        exec_time_df['exec_time'] = exec_time_df['exec_time'] / 1000000000
        
    # Create multi-metric comparison visualization
    for metric in multi_metric_df['metric'].unique():
        metric_df = multi_metric_df[multi_metric_df['metric'] == metric].copy()
        
        if metric == 'Execution Time' and not exec_time_df.empty:
            metric_df = exec_time_df
            title_suffix = " (seconds)"
        else:
            title_suffix = ""
            
        fig = px.bar(
            metric_df,
            x='query_id',
            y='value',
            hover_data=['query_sample', 'calls'],
            title=f"Top 5 Queries by {metric}{title_suffix}",
            labels={
                'query_id': 'Query ID',
                'value': metric,
                'query_sample': 'Query',
                'calls': 'Executions'
            }
        )
        fig.update_layout(height=400)
        fig.show()
        
        # Display table
        display(metric_df[['query_sample', 'value', 'calls']].rename(
            columns={
                'query_sample': 'Query',
                'value': metric,
                'calls': 'Executions'
            }
        ))
        print("\n")
else:
    print("No query data found for multiple metrics analysis.")